In [1]:
import sys
!{sys.executable} -m pip install -e ../
!{sys.executable} -m pip install addict

Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///tf/pose3D/src
  Attempting uninstall: ShAReD-Net
    Found existing installation: ShAReD-Net 1.0
    Uninstalling ShAReD-Net-1.0:
      Successfully uninstalled ShAReD-Net-1.0
  Running setup.py develop for ShAReD-Net
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
%reset -f

In [3]:
import sys

In [4]:
import time
import itertools

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [5]:
def init_devs():
    sys.setrecursionlimit(10**7) 
    
    tf.config.set_soft_device_placement(True)
    
    options = {
                "layout_optimizer": True,
                "constant_folding": True,
                "shape_optimization": True,
                "remapping": True,
                "arithmetic_optimization": True,
                "dependency_optimization": True,
                "loop_optimization": True,
                "function_optimization": True,
                "debug_stripper": False,
                "disable_model_pruning": False,
                "scoped_allocator_optimization": True,
                "pin_to_host_optimization": True,
                "implementation_selector": True,
                "disable_meta_optimizer": False
              }
    tf.config.optimizer.set_experimental_options(options)

    
    devs = tf.config.get_visible_devices()
    print(devs)

    print(tf.config.threading.get_inter_op_parallelism_threads())
    print(tf.config.threading.get_intra_op_parallelism_threads())
    tf.config.threading.set_inter_op_parallelism_threads(12)
    tf.config.threading.set_intra_op_parallelism_threads(12)
    print(tf.config.threading.get_inter_op_parallelism_threads())
    print(tf.config.threading.get_intra_op_parallelism_threads())

    gpus = tf.config.experimental.list_physical_devices('GPU')
    gpus = gpus[:] 
    if gpus:
        try:
        # Currently, memory growth needs to be the same across GPUs
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            tf.config.experimental.set_visible_devices(gpus, 'GPU')
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            # Memory growth must be set before GPUs have been initialized
            print(e)
    logical_devs = tf.config.list_logical_devices()
    physical_devs = tf.config.experimental.list_physical_devices()

    print("physical_devs",physical_devs)
    print("logical_devs", logical_devs)
    
    print(tf.version.VERSION)
init_devs()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')]
0
0
12
12
4 Physical GPUs, 4 Logical GPUs
physical_devs [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'), PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'), PhysicalDevice(name='/physical_device:XLA_GPU:1', device_type='XLA_GPU'), PhysicalDevice(name='/physical_device:XLA_GPU:2', device_type='XLA_GPU'), PhysicalDevice(name='/physical_device:XLA_GPU:3', device_type='XLA_GPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:2'

In [6]:
import ShAReD_Net.evaluation.eval_distributed as evaluate

import ShAReD_Net.evaluation.slim as evaluation_slim
import ShAReD_Net.training.slim as training_slim
import ShAReD_Net.model.slim as model_slim
import ShAReD_Net.training.loss.slim as loss_slim



In [7]:
from ShAReD_Net.configure import config

config.dataset.IMG_PATH = "/dataset/jta/images_jpg"
config.dataset.ANNO_PATH = "/dataset/jta/new_image_annotations"

config.checkpoint.path = "/tf/pose3D/checkpoints/run17_hard_batch_loss"
config.tensorboard.path = "/tf/pose3D/logdir/run17_hard_batch_loss/eval2"


In [8]:
import ShAReD_Net.data.transform.transform as transform


def create_eval_dataset(per_replica_batch_size):
    data_split = "test"
    with tf.device("/cpu:0"):
        train_ds = transform.create_dataset(data_split, 4, False).prefetch(30)
    return train_ds


In [9]:

def eval_loss():
    loss = training_slim.SlimTrainingLoss()
    return loss
    
def eval_model():
    
    low_level_extractor = model_slim.LowLevelExtractor(color_channel=13, texture_channel=16, texture_compositions=16, out_channel=32)

    encoder = model_slim.Encoder(dense_blocks_count=2, dense_filter_count=16)
    
    pos_decoder = model_slim.PosDecoder(dense_blocks_count=3, dense_filter_count=8)
    
    pose_decoder = model_slim.PoseDecoder(keypoints=config.model.output.keypoints, z_bins=config.model.z_bins, dense_blocks_count=2, dense_filter_count=16)
    
    model = evaluation_slim.SlimEvalModel(low_level_extractor, encoder, pos_decoder, pose_decoder)    
    
    return model


def create_checkpoint(eval_model, eval_loss):
    nets = {"low_level_extractor":eval_model.low_level_extractor,
            "encoder":eval_model.encoder,
            "pos_decoder":eval_model.pos_decoder,
            "pose_decoder":eval_model.pose_decoder,
           }
    ckpt = tf.train.Checkpoint(**nets)
    manager = tf.train.CheckpointManager(ckpt, config.checkpoint.path, max_to_keep=50)
    return ckpt, manager

In [10]:
import ShAReD_Net.model.modules.slim as slim_modules

roi_size_img = np.asarray(config.model.roi_size) * config.model.img_downsampling + 1
        
roi_extractor = slim_modules.Roi_Extractor(roi_size=roi_size_img)

def calc_img_index(roiindices):
    new_indexes = roiindices * config.model.img_downsampling
    return new_indexes
    

In [11]:
def eval_summery(dev, step, batch, output, loss, eval_model):
    loss_per_batch, detection_loss_sum, (loss_pos_xy_sum, loss_var_xy_sum), (loss_pos_z_sum, loss_var_z_sum) = loss    
    poses_xyz, pos_hm, (pose_prob_map_xy, pose_prob_maps_z) = output
    img, (pos_hm_gt, loss_weights), roi_indexes, (pose_xyz_gt, pose_indexes), cut_dist = batch
        
    tf.print("eval_summery")
    tf.print("Step:",step)  
    
    loss_per_batch_sum = tf.reduce_sum(loss_per_batch)
        
    tf.summary.scalar(f"detection_loss", detection_loss_sum)
    
    tf.summary.scalar(f"loss_pos_xy_sum", loss_pos_xy_sum)
    tf.summary.scalar(f"loss_pos_z_sum", loss_pos_z_sum)
    
    tf.summary.scalar(f"loss_var_xy_sum", loss_var_xy_sum)
    tf.summary.scalar(f"loss_var_z_sum", loss_var_z_sum)
        
    tf.summary.scalar(f"agg_loss", loss_per_batch_sum)
    
    for i in tf.range(tf.shape(cut_dist)[0]):
        tf.summary.scalar(f"cut_dist", cut_dist[i])
    
    for k in range(config.model.output.keypoints):
        tf.summary.scalar(f"pose x, kp {k}", tf.reduce_mean(tf.abs(poses_xyz[:,k,0]-pose_xyz_gt[:,k,0])))
        tf.summary.scalar(f"pose y, kp {k}", tf.reduce_mean(tf.abs(poses_xyz[:,k,1]-pose_xyz_gt[:,k,1])))
        tf.summary.scalar(f"pose z, kp {k}", tf.reduce_mean(tf.abs(poses_xyz[:,k,2]-pose_xyz_gt[:,k,2])))
        
    for k in range(config.model.output.keypoints):
        for i in tf.range(tf.shape(poses_xyz)[0]):
            tf.summary.scalar(f"pose x, kp {k}", poses_xyz[i,k,0])
            tf.summary.scalar(f"pose y, kp {k}", poses_xyz[i,k,1])
            tf.summary.scalar(f"pose z, kp {k}", poses_xyz[i,k,2])

            tf.summary.scalar(f"pose gt x, kp {k}", pose_xyz_gt[i,k,0])
            tf.summary.scalar(f"pose gt y, kp {k}", pose_xyz_gt[i,k,1])
            tf.summary.scalar(f"pose gt z, kp {k}", pose_xyz_gt[i,k,2])

def complex_eval_summery(dev, step, batch, output, loss, eval_model):
    poses_xyz, pos_hm, (pose_prob_map_xy, pose_prob_maps_z) = output
    img, (pos_hm_gt, loss_weights), roi_indexes, (pose_xyz_gt, pose_indexes), cut_dist  = batch
    
    tf.print("complex_eval_summery")
    
    tf.summary.image("image", img, max_outputs=4)
    
    pos_hm_gt_near = pos_hm_gt[...,0,None]
    pos_hm_gt_fare = pos_hm_gt[...,1,None]
    tf.summary.image("pos_hm_gt_near", pos_hm_gt_near, max_outputs=4)
    tf.summary.image("pos_hm_gt_fare", pos_hm_gt_fare, max_outputs=4)
    
    pos_hm_near = pos_hm[...,0,None]
    pos_hm_fare = pos_hm[...,1,None]
    tf.summary.image("pos_hm_near", pos_hm_near, max_outputs=4)
    tf.summary.image("pos_hm_fare", pos_hm_fare, max_outputs=4)
    
    new_indexes = calc_img_index(roi_indexes)
    pose_imges = roi_extractor([img, new_indexes])
    tf.summary.image("pose_imges", pose_imges, max_outputs=4)
    tf.summary.image(f"pose_hm", tf.reduce_sum(pose_prob_map_xy,axis=1), max_outputs=4)

In [12]:
def finalize(dev, step, batch, output, loss, eval_model):
    def run():
        print("Finalized")
        tf.Graph.finalize(tf.compat.v1.get_default_graph())
    tf.py_function(run,[], [])


def count_params(dev, step, batch, output, loss, eval_model):
    print(eval_model.count_params())



In [13]:
def batching(dataset, batch_size):
    batched_ds = dataset.batch(batch_size)

    def unragg(img, pos_stuff, roi_indexes, pose_stuff, cut_dist):
        rel_pose, pose_indexes = pose_stuff
        pos_heatmap, weights = pos_stuff

        def unragg_roi_indexes(indexes, row_length):
            new_indexes = np.empty([indexes.shape[0], indexes.shape[1]+1], dtype=np.int32)
            new_indexes[:,1:] = indexes
            i = 0
            b = 0
            for length in row_length[1]:
                index = indexes[i:int(i+length)]
                new_indexes[i:int(i+length),0] = b
                i += length
                b += 1
                
            return new_indexes
        
        def unragg_pose_indexes(indexes, row_length):
            new_indexes = np.empty([indexes.shape[0], indexes.shape[1], indexes.shape[-1]+1], dtype=np.int32)
            new_indexes[:,:,1:] = indexes
            i = 0
            b = 0
            for length in row_length[1]:
                index = indexes[i:int(i+length)]
                new_indexes[i:int(i+length),:,0] = b
                i += length
                b += 1
                
            return new_indexes

        roi_indexes_flat = tf.numpy_function(unragg_roi_indexes, [roi_indexes.flat_values, roi_indexes.nested_row_lengths()], Tout=roi_indexes.dtype)
        pose_indexes_flat = tf.numpy_function(unragg_pose_indexes, [pose_indexes.flat_values, pose_indexes.nested_row_lengths()], Tout=pose_indexes.dtype)

        rel_pose_flat =  rel_pose.flat_values

        return img, (pos_heatmap, weights), roi_indexes_flat, (rel_pose_flat, pose_indexes_flat), cut_dist

    unragged_ds = batched_ds.map(unragg).prefetch(100)
    return unragged_ds


In [14]:
def input_pre(batch):
    img, (pos_heatmap, weights), roi_indexes, (rel_pose, pose_indexes), cut_dist = batch
    return img, roi_indexes, pose_indexes
    
def loss_pre(output, batch):
    poses_xyz, pos_hm, (pose_prob_map_xy, pose_prob_maps_z) = output
    img, (pos_hm_gt, loss_weights), roi_indexes, (pose_xyz_gt, pose_indexes), cut_dist = batch
    return (poses_xyz, pose_xyz_gt, pose_prob_map_xy, pose_prob_maps_z), (pos_hm, pos_hm_gt, loss_weights)


lc_extra = training_slim.LossClipping(name="extra")
def grad_pre(loss, extra_loss, batch, optimizer, train_model, train_loss):
    loss_per_batch, detection_loss_sum, (loss_pos_xy_sum, loss_var_xy_sum), (loss_pos_z_sum, loss_var_z_sum) = loss    
    
    extra_loss = lc_extra(extra_loss)
    
    extra_loss_sum = tf.reduce_sum(extra_loss)
    
    loss_per_batch_sum = loss_per_batch
        
    trainable_vars =  train_model.low_level_extractor.trainable_variables + train_model.encoder.trainable_variables + train_model.pos_decoder.trainable_variables + train_model.pose_decoder.trainable_variables+ train_loss.trainable_variables
        
    return loss_per_batch, extra_loss_sum, trainable_vars



In [15]:
dist_strat = tf.distribute.MirroredStrategy(cross_device_ops = tf.distribute.HierarchicalCopyAllReduce())

steps = 100000

step_callbacks = evaluate.standart_callbacks()
step_callbacks.every_steps[1] = eval_summery
step_callbacks.every_steps[10] = complex_eval_summery

step_callbacks.at_step[1] = count_params
step_callbacks.at_step[2] = finalize

step_callbacks.make_batches = batching

step_callbacks.grad_pre = grad_pre
step_callbacks.input_pre = input_pre
step_callbacks.loss_pre = loss_pre

step_callbacks.create_dataset = create_eval_dataset

step_callbacks.create_ckpt = create_checkpoint

step_callbacks.create_loss = eval_loss

step_callbacks.create_model = eval_model


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')


In [ ]:
evaluate.evaluate(steps, dist_strat, batch_size = config.training.batch_size, learning_rate=config.training.learning_rate, callbacks = step_callbacks)


Roi_Extractor [TensorShape([1, None, None, 3]), TensorShape([None, 3])]
Restored from /tf/pose3D/checkpoints/run17_hard_batch_loss/ckpt-30250
Instructions for updating:
renamed to `run`
SlimEvalModel (TensorShape([None, 421, 421, 3]), TensorShape(None), TensorShape(None))
Extractor (None, None, None, 3)
Encoder (None, None, None, 32)
stage1 [TensorShape([None, None, None, 32]), TensorShape([None, None, None, 32])]
big_shared1 [TensorShape([None, None, None, 32]), TensorShape([None, None, None, 32])]
ResAttention [TensorShape([None, None, None, 32]), TensorShape([None, None, None, 32])]
Attention [TensorShape([None, None, None, 32]), TensorShape([None, None, None, 32])]
ShReD [TensorShape([None, None, None, 32]), TensorShape([None, None, None, 96])]
DenseModule (None, None, None, 128)
DenseBlock (None, None, None, 24)
BnDoConfReluConfRelu (None, None, None, 24)
DenseBlock (None, None, None, 40)
BnDoConfReluConfRelu (None, None, None, 40)
big_normal (None, None, None, 56)
normal_shared1 

DenseBlock (None, None, None, 40)
BnDoConfReluConfRelu (None, None, None, 40)
scale_1_4_res (None, None, None, 32)
scale_1_4_shc (None, None, None, 56)
stage3 [TensorShape([None, None, None, 64]), TensorShape([None, None, None, 56])]
big_shared1 [TensorShape([None, None, None, 64]), TensorShape([None, None, None, 56])]
ResAttention [TensorShape([None, None, None, 64]), TensorShape([None, None, None, 56])]
Attention [TensorShape([None, None, None, 64]), TensorShape([None, None, None, 56])]
ShReD [TensorShape([None, None, None, 64]), TensorShape([None, None, None, 192])]
DenseModule (None, None, None, 256)
DenseBlock (None, None, None, 24)
BnDoConfReluConfRelu (None, None, None, 24)
DenseBlock (None, None, None, 40)
BnDoConfReluConfRelu (None, None, None, 40)
big_normal (None, None, None, 56)
normal_shared1 [TensorShape([None, None, None, 64]), TensorShape([None, None, None, 56])]
scale_down (None, None, None, 64)
ResAttention [TensorShape([None, None, None, 64]), TensorShape([None, None

small_medium (None, None, None, 48)
medium_shared3 [TensorShape([None, None, None, 64]), TensorShape([None, None, None, 96])]
scale_down (None, None, None, 64)
ResAttention [TensorShape([None, None, None, 64]), TensorShape([None, None, None, 96])]
Attention [TensorShape([None, None, None, 64]), TensorShape([None, None, None, 96])]
DenseModule (None, None, None, 256)
DenseBlock (None, None, None, 24)
BnDoConfReluConfRelu (None, None, None, 24)
DenseBlock (None, None, None, 32)
BnDoConfReluConfRelu (None, None, None, 32)
DenseBlock (None, None, None, 40)
BnDoConfReluConfRelu (None, None, None, 40)
scale_up (None, None, None, 48)
medium_normal (None, None, None, 48)
normal_shared3 [TensorShape([None, None, None, 64]), TensorShape([None, None, None, 96])]
scale_down (None, None, None, 64)
ResAttention [TensorShape([None, None, None, 64]), TensorShape([None, None, None, 96])]
Attention [TensorShape([None, None, None, 64]), TensorShape([None, None, None, 96])]
DenseModule (None, None, None, 

DenseBlock (None, 35, 35, 24)
BnDoConfReluConfRelu (None, 35, 35, 24)
DenseBlock (None, 35, 35, 40)
BnDoConfReluConfRelu (None, 35, 35, 40)
SelfShAReD [TensorShape([None, 35, 35, 70]), TensorShape([None, 35, 35, 56])]
ShAReD [TensorShape([None, 35, 35, 70]), TensorShape([None, 35, 35, 56])]
ResAttention [TensorShape([None, 35, 35, 70]), TensorShape([None, 35, 35, 56])]
Attention [TensorShape([None, 35, 35, 70]), TensorShape([None, 35, 35, 56])]
ShReD [TensorShape([None, 35, 35, 70]), TensorShape([None, 35, 35, 210])]
DenseModule (None, 35, 35, 280)
DenseBlock (None, 35, 35, 24)
BnDoConfReluConfRelu (None, 35, 35, 24)
DenseBlock (None, 35, 35, 40)
BnDoConfReluConfRelu (None, 35, 35, 40)
Attention [TensorShape([None, 35, 35, 56]), TensorShape([None, 35, 35, 56])]
ShAReD [TensorShape([None, 35, 35, 70]), TensorShape([None, 35, 35, 112])]
ResAttention [TensorShape([None, 35, 35, 70]), TensorShape([None, 35, 35, 112])]
Attention [TensorShape([None, 35, 35, 70]), TensorShape([None, 35, 35, 1

Step: 57
eval_summery
Step: 57
eval_summery
Step: 57
eval_summery
Step: 57
eval_summery
Step: 58
eval_summery
Step: 58
eval_summery
Step: 58
eval_summery
Step: 58
eval_summery
Step: 59
eval_summery
Step: 59
eval_summery
Step: 59
eval_summery
Step: 59
eval_summery
Step: 60
complex_eval_summery
eval_summery
Step: 60
complex_eval_summery
eval_summery
Step: 60
complex_eval_summery
eval_summery
Step: 60
complex_eval_summery
eval_summery
Step: 61
eval_summery
Step: 61
eval_summery
Step: 61
eval_summery
Step: 61
eval_summery
Step: 62
eval_summery
Step: 62
eval_summery
Step: 62
eval_summery
Step: 62
eval_summery
Step: 63
eval_summery
Step: 63
eval_summery
Step: 63
eval_summery
Step: 63
eval_summery
Step: 64
eval_summery
Step: 64
eval_summery
Step: 64
eval_summery
Step: 64
eval_summery
Step: 65
eval_summery
Step: 65
eval_summery
Step: 65
eval_summery
Step: 65
eval_summery
Step: 66
eval_summery
Step: 66
eval_summery
Step: 66
eval_summery
Step: 66
eval_summery
Step: 67
eval_summery
Step: 67
eval_

...

eval_summery
Step: 4334
eval_summery
Step: 4334
eval_summery
Step: 4335
eval_summery
Step: 4335
eval_summery
Step: 4335
eval_summery
Step: 4335
eval_summery
Step: 4336
eval_summery
Step: 4336
eval_summery
Step: 4336
eval_summery
Step: 4336
eval_summery
Step: 4337
eval_summery
Step: 4337
eval_summery
Step: 4337
eval_summery
Step: 4337
eval_summery
Step: 4338
eval_summery
Step: 4338
eval_summery
Step: 4338
eval_summery
Step: 4338
eval_summery
Step: 4339
eval_summery
Step: 4339
eval_summery
Step: 4339
eval_summery
Step: 4339
eval_summery
Step: 4340
complex_eval_summery
eval_summery
Step: 4340
complex_eval_summery
eval_summery
Step: 4340
complex_eval_summery
eval_summery
Step: 4340
complex_eval_summery
eval_summery
Step: 4341
eval_summery
Step: 4341
eval_summery
Step: 4341
eval_summery
Step: 4341
eval_summery
Step: 4342
eval_summery
Step: 4342
eval_summery
Step: 4342
eval_summery
Step: 4342
eval_summery
Step: 4343
eval_summery
Step: 4343
eval_summery
Step: 4343
eval_summery
Step: 4343
eval

Step: 4412
eval_summery
Step: 4413
eval_summery
Step: 4413
eval_summery
Step: 4413
eval_summery
Step: 4413
eval_summery
Step: 4414
eval_summery
Step: 4414
eval_summery
Step: 4414
eval_summery
Step: 4414
eval_summery
Step: 4415
eval_summery
Step: 4415
eval_summery
Step: 4415
eval_summery
Step: 4415
eval_summery
Step: 4416
eval_summery
Step: 4416
eval_summery
Step: 4416
eval_summery
Step: 4416
eval_summery
Step: 4417
eval_summery
Step: 4417
eval_summery
Step: 4417
eval_summery
Step: 4417
eval_summery
Step: 4418
eval_summery
Step: 4418
eval_summery
Step: 4418
eval_summery
Step: 4418
eval_summery
Step: 4419
eval_summery
Step: 4419
eval_summery
Step: 4419
eval_summery
Step: 4419
eval_summery
Step: 4420
complex_eval_summery
eval_summery
Step: 4420
complex_eval_summery
eval_summery
Step: 4420
complex_eval_summery
eval_summery
Step: 4420
complex_eval_summery
eval_summery
Step: 4421
eval_summery
Step: 4421
eval_summery
Step: 4421
eval_summery
Step: 4421
eval_summery
Step: 4422
eval_summery
Step

eval_summery
Step: 4491
eval_summery
Step: 4491
eval_summery
Step: 4491
eval_summery
Step: 4492
eval_summery
Step: 4492
eval_summery
Step: 4492
eval_summery
Step: 4492
eval_summery
Step: 4493
eval_summery
Step: 4493
eval_summery
Step: 4493
eval_summery
Step: 4493
eval_summery
Step: 4494
eval_summery
Step: 4494
eval_summery
Step: 4494
eval_summery
Step: 4494
eval_summery
Step: 4495
eval_summery
Step: 4495
eval_summery
Step: 4495
eval_summery
Step: 4495
eval_summery
Step: 4496
eval_summery
Step: 4496
eval_summery
Step: 4496
eval_summery
Step: 4496
eval_summery
Step: 4497
eval_summery
Step: 4497
eval_summery
Step: 4497
eval_summery
Step: 4497
eval_summery
Step: 4498
eval_summery
Step: 4498
eval_summery
Step: 4498
eval_summery
Step: 4498
eval_summery
Step: 4499
eval_summery
Step: 4499
eval_summery
Step: 4499
eval_summery
Step: 4499
eval_summery
Step: 4500
complex_eval_summery
eval_summery
Step: 4500
complex_eval_summery
eval_summery
Step: 4500
complex_eval_summery
eval_summery
Step: 4500
c

eval_summery
Step: 4570
complex_eval_summery
eval_summery
Step: 4570
complex_eval_summery
eval_summery
Step: 4570
complex_eval_summery
eval_summery
Step: 4571
eval_summery
Step: 4571
eval_summery
Step: 4571
eval_summery
Step: 4571
eval_summery
Step: 4572
eval_summery
Step: 4572
eval_summery
Step: 4572
eval_summery
Step: 4572
eval_summery
Step: 4573
eval_summery
Step: 4573
eval_summery
Step: 4573
eval_summery
Step: 4573
eval_summery
Step: 4574
eval_summery
Step: 4574
eval_summery
Step: 4574
eval_summery
Step: 4574
eval_summery
Step: 4575
eval_summery
Step: 4575
eval_summery
Step: 4575
eval_summery
Step: 4575
eval_summery
Step: 4576
eval_summery
Step: 4576
eval_summery
Step: 4576
eval_summery
Step: 4576
eval_summery
Step: 4577
eval_summery
Step: 4577
eval_summery
Step: 4577
eval_summery
Step: 4577
eval_summery
Step: 4578
eval_summery
Step: 4578
eval_summery
Step: 4578
eval_summery
Step: 4578
eval_summery
Step: 4579
eval_summery
Step: 4579
eval_summery
Step: 4579
eval_summery
Step: 4579
e

Step: 4648
eval_summery
Step: 4649
eval_summery
Step: 4649
eval_summery
Step: 4649
eval_summery
Step: 4649
eval_summery
Step: 4650
complex_eval_summery
eval_summery
Step: 4650
complex_eval_summery
eval_summery
Step: 4650
complex_eval_summery
eval_summery
Step: 4650
complex_eval_summery
eval_summery
Step: 4651
eval_summery
Step: 4651
eval_summery
Step: 4651
eval_summery
Step: 4651
eval_summery
Step: 4652
eval_summery
Step: 4652
eval_summery
Step: 4652
eval_summery
Step: 4652
eval_summery
Step: 4653
eval_summery
Step: 4653
eval_summery
Step: 4653
eval_summery
Step: 4653
eval_summery
Step: 4654
eval_summery
Step: 4654
eval_summery
Step: 4654
eval_summery
Step: 4654
eval_summery
Step: 4655
eval_summery
Step: 4655
eval_summery
Step: 4655
eval_summery
Step: 4655
eval_summery
Step: 4656
eval_summery
Step: 4656
eval_summery
Step: 4656
eval_summery
Step: 4656
eval_summery
Step: 4657
eval_summery
Step: 4657
eval_summery
Step: 4657
eval_summery
Step: 4657
eval_summery
Step: 4658
eval_summery
Step

estimator_loss_z 228960.422
estimator_loss_xy 15820.8545

detection_loss 98.055069


estimator_loss_var_z 15143.8232
estimator_loss_var_xy 124911

cp 606